
## New York City 2022 Fiscal Year Salary Prediction

In [0]:
from pyspark.sql.types import  StructField, StructType, StringType, IntegerType,DateType, FloatType, DoubleType

In [0]:
filePath = "/FileStore/tables/Citywide_Payroll_Data__Fiscal_Year_-1.csv"

rawDF = spark.read.csv(filePath, header="true", inferSchema="true", multiLine="true", escape='"')

display(rawDF.head())

Row(Fiscal Year=2020, Payroll Number=17, Agency Name='OFFICE OF EMERGENCY MANAGEMENT', Last Name='BEREZIN', First Name='MIKHAIL', Mid Init=None, Agency Start Date=datetime.date(2015, 8, 10), Work Location Borough='BROOKLYN', Title Description='EMERGENCY PREPAREDNESS MANAGER', Leave Status as of June 30='ACTIVE', Base Salary=86005.0, Pay Basis='per Annum', Regular Hours=1820.0, Regular Gross Paid=84698.21, OT Hours=0.0, Total OT Paid=0.0, Total Other Pay=0.0)

### Data Cleaning


In [0]:
from pyspark.sql.functions import *

In [0]:
rawDF = rawDF.na.drop('any')

In [0]:
yeardf = rawDF.withColumn("Year", year(rawDF["Agency Start Date"]))

In [0]:
diffdf = yeardf.withColumn("Years_Working", col("Fiscal Year") - col("Year"))

In [0]:
hoursdf = diffdf[(diffdf['Regular Hours'] > 0) & (diffdf['OT Hours'] > 0) & (diffdf['Total OT Paid'] > 0) & (diffdf['Total Other Pay'] > 0) ]

In [0]:
presentdf = hoursdf[(hoursdf['Leave Status as of June 30'] == 'ACTIVE')]

In [0]:
presentdf.groupBy("Work Location Borough") \
         .count() \
         .orderBy(desc("count")) \
         .limit(10) \
         .show()

+---------------------+------+
|Work Location Borough| count|
+---------------------+------+
|            MANHATTAN|128683|
|               QUEENS|103441|
|             BROOKLYN| 97448|
|                BRONX| 54504|
|             RICHMOND| 18342|
|          WESTCHESTER|   869|
|               ULSTER|   353|
|             SULLIVAN|   226|
|             DELAWARE|   190|
|               PUTNAM|    59|
+---------------------+------+



In [0]:
rows = ["MANHATTAN", "QUEENS", "BROOKLYN", "BRONX", "RICHMOND"]

In [0]:
loc_df = presentdf.filter(col("Work Location Borough").isin(rows))

In [0]:
presentdf.groupBy("Agency Name") \
         .count() \
         .orderBy(desc("count")) \
         .limit(15) \
         .show()

+--------------------+------+
|         Agency Name| count|
+--------------------+------+
|   POLICE DEPARTMENT|158006|
|     FIRE DEPARTMENT| 59648|
|DEPARTMENT OF SAN...| 32321|
|DEPARTMENT OF COR...| 22034|
|DEPT OF PARKS & R...| 17480|
|NYC HOUSING AUTHO...| 17044|
|HRA/DEPT OF SOCIA...| 14602|
|ADMIN FOR CHILDRE...| 13969|
|DEPARTMENT OF TRA...| 12787|
|DEPARTMENT OF EDU...|  9795|
|DEPT OF ENVIRONME...|  9578|
|DEPT OF HEALTH/ME...|  7875|
|DEPT. OF HOMELESS...|  4852|
|DEPT OF CITYWIDE ...|  3557|
|DEPARTMENT OF BUI...|  2370|
+--------------------+------+



In [0]:
agency_rows = ["POLICE DEPARTMENT","FIRE DEPARTMENT","DEPARTMENT OF SANITATION","DEPARTMENT OF CORRECTION","DEPT OF PARKS & RECREATION","NYC HOUSING AUTHORITY"]

In [0]:
agency_df = loc_df.filter(col("Agency Name").isin(agency_rows))

In [0]:
presentdf.groupBy("Title Description") \
         .count() \
         .orderBy(desc("count")) \
         .limit(15) \
         .show()

+--------------------+-----+
|   Title Description|count|
+--------------------+-----+
|      POLICE OFFICER|78588|
|         FIREFIGHTER|32578|
|   SANITATION WORKER|22572|
|  CORRECTION OFFICER|16810|
| SCHOOL SAFETY AGENT|14556|
|           SERGEANT-|13621|
|     P.O. DA DET GR3|12316|
|          LIEUTENANT|11245|
|EMERGENCY MEDICAL...| 8934|
|           CARETAKER| 7317|
|CHILD PROTECTIVE ...| 6660|
| COMMUNITY ASSOCIATE| 6213|
|             CAPTAIN| 5200|
|COMMUNITY COORDIN...| 4941|
|TRAFFIC ENFORCEME...| 4501|
+--------------------+-----+



In [0]:
title_rows = ["POLICE OFFICER", "FIREFIGHTER", "SANITATION WORKER", "CORRECTION OFFICER", "SCHOOL SAFETY AGENT","SERGEANT-","P.O. DA DET GR3","LIEUTENANT","EMERGENCY MEDICAL SPECIALIST-EMT","CARETAKER","CAPTAIN"]

In [0]:
job_df = agency_df.filter(col("Title Description").isin(title_rows))

In [0]:
display(job_df.summary())

summary,Fiscal Year,Payroll Number,Agency Name,Last Name,First Name,Mid Init,Work Location Borough,Title Description,Leave Status as of June 30,Base Salary,Pay Basis,Regular Hours,Regular Gross Paid,OT Hours,Total OT Paid,Total Other Pay,Year,Years_Working
count,223590,223590,223590,223590,223590,223590,223590,223590,223590,223590,223590,223590,223590,223590,223590,223590,223590,223590
mean,2019.9617693098976,166.16562458070575,null,null,NaN,null,null,null,null,75378.97500782683,null,2043.7847135406892,73918.59835077709,354.599840690548,18073.957837738617,12584.375584550891,2009.2992665146026,10.662502795294959
stddev,1.405692076336649,277.1495713967898,null,null,NaN,null,null,null,null,23740.0365050578,null,152.73283285877068,25501.21536604218,263.43931861633865,14975.625476924466,7532.994936903355,7.241558228058092,7.125440580287705
min,2018,56,DEPARTMENT OF CORRECTION,AADAMS,A,A,BRONX,CAPTAIN,ACTIVE,29636.0,per Annum,125.72,-684.53,0.08,1.89,0.02,1966,0
25%,2019,56,null,null,NaN,null,null,null,null,50207.0,null,2045.72,49635.86,159.3,6479.04,5903.99,2004,4
50%,2020,56,null,null,NaN,null,null,null,null,85292.0,null,2080.0,84346.17,305.1,14186.75,13480.84,2010,10
75%,2021,57,null,null,NaN,null,null,null,null,85292.0,null,2080.0,85058.23,482.48,26481.92,18017.72,2016,16
max,2022,996,POLICE DEPARTMENT,ZYTKOWICZ,ZYNAISHA,Z,RICHMOND,SERGEANT-,ACTIVE,176449.0,per Annum,2475.4,186505.23,3340.77,188594.12,99512.69,2022,54


In [0]:
zero_df = job_df[(job_df['Regular Gross Paid'] > 0)]

In [0]:
dup_df = zero_df.dropDuplicates()

In [0]:
columnsToKeep = ["Work Location Borough","Agency Name","Title Description","Regular Hours","OT Hours","Total OT Paid","Total Other Pay", "Years_Working","Regular Gross Paid"]

df = dup_df.select(columnsToKeep)

In [0]:
df.cache().count()
display(df.head())

Row(Work Location Borough='MANHATTAN', Agency Name='POLICE DEPARTMENT', Title Description='LIEUTENANT', Regular Hours=2080.0, OT Hours=750.75, Total OT Paid=52691.31, Total Other Pay=21609.12, Years_Working=28, Regular Gross Paid=131514.59)

### Visualizations 


In [0]:
display(df.groupBy("Years_Working").agg(avg("Regular Gross Paid").alias("Avg_Salary")).orderBy(col("Avg_Salary").desc()))


Years_Working,Avg_Salary
43,130762.79
42,115751.53222222223
40,109435.44341463414
41,109106.24818181815
39,108437.4842857143
37,108027.85376237628
36,107608.2099218751
35,105031.13360465114
38,104646.48484848479
27,102477.10900347619


Databricks visualization. Run in Databricks to view.

In [0]:
display(df.groupBy("Agency Name").agg(avg("Regular Gross Paid").alias("Avg_Salary")).orderBy(col("Avg_Salary").desc()))


Agency Name,Avg_Salary
FIRE DEPARTMENT,77666.7815768527
DEPARTMENT OF CORRECTION,75723.51377640612
POLICE DEPARTMENT,75645.59088981565
DEPARTMENT OF SANITATION,66691.18563707253
NYC HOUSING AUTHORITY,36425.094031706976


Databricks visualization. Run in Databricks to view.

In [0]:
display(df.groupBy("Work Location Borough").agg(avg("Regular Gross Paid").alias("Avg_Salary")).orderBy(col("Avg_Salary").desc()))


Work Location Borough,Avg_Salary
MANHATTAN,78694.98970116806
RICHMOND,76563.48916805787
QUEENS,73068.7869773628
BROOKLYN,71271.58703179717
BRONX,70571.50640989789


Databricks visualization. Run in Databricks to view.

In [0]:
display(df.groupBy("Title Description").agg(avg("Regular Gross Paid").alias("Avg_Salary")).orderBy(col("Avg_Salary").desc()))


Title Description,Avg_Salary
CAPTAIN,117025.41448644374
LIEUTENANT,114329.11427034235
SERGEANT-,105090.68398722565
P.O. DA DET GR3,97554.02021516723
FIREFIGHTER,77614.2160556204
CORRECTION OFFICER,72951.42560440215
POLICE OFFICER,69578.52396770465
SANITATION WORKER,66691.18563707253
EMERGENCY MEDICAL SPECIALIST-EMT,44540.60889411239
SCHOOL SAFETY AGENT,42936.641086837044


Databricks visualization. Run in Databricks to view.

In [0]:
table_name = "FISCAL_SALARY"
#noNullsDF.write.format("parquet").saveAsTable(table_name)

### Preprocessing Data


In [0]:
trainDF, testDF = df.randomSplit([.8, .2], seed=42)

In [0]:
print(testDF.count())
print(trainDF.count())

44616
178973


In [0]:
from pyspark.ml.feature import StringIndexer, VectorAssembler

categoricalCols = [field for (field, dataType) in trainDF.dtypes if dataType == "string"]

indexOutputCols = [x + "Index" for x in categoricalCols]
stringIndexer = StringIndexer(inputCols=categoricalCols, outputCols=indexOutputCols, handleInvalid="skip")

numericCols = [field for (field, dataType) in trainDF.dtypes if ((dataType == "double") & (field != "Gross Regular Paid"))]

assemblerInputs = indexOutputCols + numericCols
vecAssembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

### Gradient Boosted Trees Regression

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

In [0]:
gbt = GBTRegressor(labelCol="Regular Gross Paid", maxIter=10, featuresCol="features")

In [0]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[stringIndexer,vecAssembler, gbt])


In [0]:
model = pipeline.fit(trainDF)

In [0]:
pred= model.transform(testDF)

In [0]:
pred.select("features", "Regular Gross Paid", "prediction").show()

+--------------------+------------------+------------------+
|            features|Regular Gross Paid|        prediction|
+--------------------+------------------+------------------+
|[3.0,3.0,3.0,1952...|           79532.7| 78788.80092707623|
|[3.0,3.0,3.0,2080...|           66949.1| 67276.81760499463|
|[3.0,2.0,2.0,1405...|           24998.8| 26736.25681986374|
|[3.0,2.0,2.0,2080...|          75836.92| 74272.00506463072|
|[3.0,2.0,2.0,2085...|          41096.53| 40589.49949711799|
|[3.0,1.0,8.0,705....|          12607.58|15185.352978628509|
|[3.0,1.0,8.0,2239...|          46380.46| 46087.76450023704|
|[3.0,1.0,1.0,2045...|          85058.22| 84910.08515817893|
|[3.0,1.0,1.0,2080...|          54838.72| 53515.18494073205|
|[3.0,1.0,1.0,2080...|          84831.14| 84800.27283982863|
|[3.0,1.0,1.0,2080...|          107815.2|105584.88409309446|
|[3.0,1.0,1.0,2080...|          72201.24| 69985.63745284909|
|[3.0,1.0,1.0,2080...|          85051.84| 84934.91966111492|
|[3.0,1.0,1.0,2080...|  

In [0]:
from pyspark.ml.tuning import ParamGridBuilder
paramGrid = ParamGridBuilder().addGrid(gbt.maxDepth, [2, 5]).addGrid(gbt.maxIter, [10, 100]).build()

evaluator = RegressionEvaluator(labelCol="Regular Gross Paid", predictionCol="prediction", metricName="rmse")

rmse = evaluator.evaluate(pred)
r2 = evaluator.setMetricName("r2").evaluate(pred)

print(rmse)
print(r2)

1531.3997480772275
0.9963841700338091


In [0]:
import pyspark.sql.functions as F

residuals = pred.withColumn("residual", (F.col("Regular Gross Paid") - F.col("prediction")))
display(residuals.agg({'residual': 'mean'}))

avg(residual)
-1.5446005075490725


In [0]:
display(residuals.select("prediction", "residual").limit(10))

prediction,residual
81063.53292932124,-1530.8329293212446
73402.04629638793,-6452.94629638792
27215.88303701086,-2217.0830370108597
76607.77316028628,-770.8531602862786
44515.82719220669,-3419.2971922066936
17727.80255808838,-5120.2225580883805
46031.18789976569,349.2721002343096
85626.48860629727,-568.2686062972643
55372.622225957966,-533.9022259579651
85860.06621383007,-1028.9262138300692


Databricks visualization. Run in Databricks to view.


### Random Forest Regression

In [0]:
from pyspark.ml.regression import RandomForestRegressor
rf = RandomForestRegressor(labelCol="Regular Gross Paid", maxBins=40, seed=42)
pipeline = Pipeline(stages = [stringIndexer, vecAssembler, rf])

In [0]:
model = pipeline.fit(trainDF)

In [0]:
pred= model.transform(testDF)

In [0]:
evaluator = RegressionEvaluator(labelCol="Regular Gross Paid", predictionCol="prediction", metricName="rmse")

rmse = evaluator.evaluate(pred)
r2 = evaluator.setMetricName("r2").evaluate(pred)

print(rmse)
print(r2)

3508.2342378381754
0.9810238535975079


In [0]:
display(residuals.select("prediction", "residual").limit(10))

prediction,residual
78788.80092707623,743.8990729237703
67276.81760499463,-327.71760499462835
26736.25681986374,-1737.456819863739
74272.00506463072,1564.9149353692774
40589.49949711799,507.03050288200757
15185.352978628509,-2577.772978628509
46087.76450023704,292.6954997629582
84910.08515817893,148.13484182106913
53515.18494073205,1323.5350592679533
84800.27283982863,30.867160171372234


In [0]:
import pyspark.sql.functions as F

residuals = pred.withColumn("residual", (F.col("Regular Gross Paid") - F.col("prediction")))
display(residuals.agg({'residual': 'mean'}))

avg(residual)
-3.7160203827914913


In [0]:
display(residuals.select("prediction", "residual").limit(10))

prediction,residual
81063.53292932124,-1530.8329293212446
73402.04629638793,-6452.94629638792
27215.88303701086,-2217.0830370108597
76607.77316028628,-770.8531602862786
44515.82719220669,-3419.2971922066936
17727.80255808838,-5120.2225580883805
46031.18789976569,349.2721002343096
85626.48860629727,-568.2686062972643
55372.622225957966,-533.9022259579651
85860.06621383007,-1028.9262138300692


Databricks visualization. Run in Databricks to view.

In [0]:
model.stages[-1].featureImportances

Out[50]: SparseVector(8, {0: 0.0, 1: 0.0146, 2: 0.0944, 3: 0.0189, 4: 0.0012, 5: 0.024, 6: 0.3197, 7: 0.5272})

In [0]:
featureImportance = model.stages[-1].featureImportances.toArray()
featureNames = map(lambda s: s.name, df.schema.fields)
featureImportanceMap = zip(featureImportance, featureNames)

In [0]:
importancesDf = spark.createDataFrame(sc.parallelize(featureImportanceMap).map(lambda r: [r[1], float(r[0])]))

In [0]:
Feature_DF = importancesDf.withColumnRenamed("_1", "Features").withColumnRenamed("_2", "Importance")

display(Feature_DF.orderBy(desc("Importance")))

Features,Importance
Years_Working,0.5271753989933409
Total Other Pay,0.3197387531659057
Title Description,0.09444600402932907
Total OT Paid,0.023960120016458114
Regular Hours,0.018852009760189827
Agency Name,0.014596763553701228
OT Hours,0.0012255581540514216
Work Location Borough,5.392327023854675E-6


Databricks visualization. Run in Databricks to view.